In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [5]:
pd.options.display.max_columns=None

In [6]:
df_esios=pd.read_csv('../data/datos_esios.csv', sep=';')

In [7]:
df_esios.head()

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1
0,2015-10-01 00:00:00,2196.1,24517.1,8203.3,5355.4,4705.3,56.65,38.56,56.31
1,2015-10-01 01:00:00,2064.4,23168.6,8286.8,5355.4,4705.3,53.50,31.27,54.25
2,2015-10-01 02:00:00,1866.5,22437.2,8301.3,5355.4,4705.3,49.69,30.66,49.69
3,2015-10-01 03:00:00,1660.9,22281.3,8281.3,5355.4,4705.3,49.10,29.14,49.10
4,2015-10-01 04:00:00,1373.9,21909.7,8247.7,5355.4,4705.3,51.25,29.40,51.25


In [8]:
df_esios.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1
count,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.00000
mean,5479.286014,28672.990169,4409.936025,4420.339625,4188.581794,48.739281,42.105033,48.46574
std,3185.927399,4616.496856,2216.248299,474.038209,469.363759,14.954689,20.799903,15.09524
min,240.800000,17728.400000,546.000000,2461.500000,2315.100000,2.060000,-31.820000,0.00000
25%,3018.400000,24734.150000,2316.975000,4125.800000,3797.600000,40.137500,29.740000,40.05000
50%,4822.950000,28920.600000,4634.000000,4483.200000,4148.100000,49.550000,39.165000,49.00000
75%,7289.250000,32267.650000,6282.750000,4799.700000,4705.300000,59.450000,51.562500,58.90000
max,16952.600000,41297.900000,8768.300000,9481.400000,8270.600000,101.990000,874.010000,102.49000


In [11]:
df_esios['datetime'].describe()  #tenemos repetidos los datos para el dia de 25 horas.

count                   26304
unique                  26301
top       2015-10-25 02:00:00
freq                        2
Name: datetime, dtype: object

#### 1) Calculamos columna a predecir:

"reservar"

* 1: Baja intra (PMI1<=PMD)  -->¿Separar? Sube baja igual
* 0: Sube intra (PMI1>0)

Creamos subconjunto con precio md, precio mi1


In [27]:
df_esios_precio=df_esios[['PRECIO_MD_ESP','PRECIO_I1']]

In [28]:
df_esios_precio.head()

,PRECIO_MD_ESP,PRECIO_I1
0,56.65,56.31
1,53.50,54.25
2,49.69,49.69
3,49.10,49.10
4,51.25,51.25
